In [ ]:
import tarfile
import numpy as np
import re
from glob import glob
import pandas as pd
import pickle
import os
import wget
import tensorflow as tf
from IPython import display
import matplotlib.pyplot as plt
from mammo_utils import extract_tar, read_pgm, download_file


# Batch generator
def get_batches(X, y, batch_size, distort=True):
    # Shuffle X,y
    shuffled_idx = np.arange(len(y))
    np.random.shuffle(shuffled_idx)
    i, h, w, c = X.shape
    
    # Enumerate indexes by steps of batch_size
    for i in range(0, len(y), batch_size):
        batch_idx = shuffled_idx[i:i+batch_size]
        X_return = X[batch_idx]
        
        # do random flipping of images
        coin = np.random.binomial(1, 0.5, size=None)
        if coin and distort:
            X_return = X_return[...,::-1,:]
        
        yield X_return, y[batch_idx]
        
def read_queue(filename_queue, batch_size=32):
    feature = {'image': tf.FixedLenFeature([], tf.string), 
               'label_class': tf.FixedLenFeature([], tf.int64),
               'label_normal': tf.FixedLenFeature([], tf.int64),
               'label_pathology': tf.FixedLenFeature([], tf.int64),
               'label_type': tf.FixedLenFeature([], tf.int64),
              }

    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)

    features = tf.parse_single_example(serialized_example, features=feature)
 
    #Convert the image data from string back to the numbers
    image = tf.decode_raw(features['image'], tf.uint8)

    # scale the image
    image = tf.to_float(image)
    image = tf.divide(tf.subtract(image, 128), 255.0)

    # Cast label data into int32
    label = tf.cast(features['label_class'], tf.int64)

    # Reshape image data into the original shape
    image = tf.reshape(image, [299, 299, 1])

    images, labels = tf.train.shuffle_batch([image, label], batch_size=batch_size, capacity=30, num_threads=1, min_after_dequeue=10)

    return images, labels

In [ ]:
train_labels = np.load(os.path.join("data", "all_mias_labels_train.npy"))
test_labels = np.load(os.path.join("data", "all_mias_labels_test.npy"))

train_images = np.load(os.path.join("data","all_mias_slices_train.npy"))
test_images = np.load(os.path.join("data","all_mias_slices_test.npy"))

In [ ]:
# encode the labels
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(train_labels)
y_tr = le.transform(train_labels)
y_cv = le.transform(test_labels)

y_all = np.concatenate([y_tr, y_cv], axis=0)

# pick how to classify the data
#y_tr = labels.ABNORMAL.values
num_classes = len(np.unique(y_tr))

# scale the data
X_tr = (train_images - 128) / 255.0
X_cv = (test_images - 128) / 255.0

X_all = np.concatenate([X_tr, X_cv], axis=0)

from sklearn.model_selection import train_test_split

# Split data into training and test
X_tr, X_cv, y_tr, y_cv = train_test_split(X_all, y_all, test_size=0.2, random_state=3)

print("X_tr:", X_tr.shape)
print("y_tr:", y_tr.shape)
print("X_cv:", X_cv.shape)
print("y_cv:", y_cv.shape)

print("\nValidation distribution:\n",pd.value_counts(y_cv, normalize=True))
print("\nTraining distribution:\n",pd.value_counts(y_tr, normalize=True))

print("\nClasses:", le.classes_)
print("Num Classes:", num_classes)

In [ ]:
batch_size = 10
num_threads = 10
min_after_dequeue = 10000
capacity = min_after_dequeue + (num_threads + 1) * batch_size

def create_batches(file_names, labels, batch_size=32):
    # create tensors
    labels = tf.convert_to_tensor(labels, dtype=tf.int32)
    files = tf.convert_to_tensor(file_names, dtype=tf.string)
    
    # create the queue
    file_names_q, label_q = tf.train.slice_input_producer([files, labels])
    
    # read the data
    image_queue = tf.read_file(file_names_q)  # convert filenames to content
    image_queue = tf.image.decode_jpeg(image_queue, channels=3)
    image_queue.set_shape([512, 512, 3])
    image_queue = tf.to_float(image_queue)  # convert uint8 to float32
    
    return tf.train.shuffle_batch([image_queue, label_q], batch_size, capacity, min_after_dequeue, num_threads=num_threads)

In [ ]:
# config
epochs = 1                 
batch_size = 32

# learning rate decay variables
steps_per_epoch = (1120 * 4) / batch_size

## Hyperparameters
# Small epsilon value for the BN transform
epsilon = 1e-8

# learning rate
epochs_per_decay = 10
starting_rate = 0.003
decay_factor = 0.85
staircase = True

# lambdas
lamC = 0.00005
lamF = 0.00100

# use dropout
dropout = False

In [ ]:
graph = tf.Graph()
# whether to retrain model from scratch or use saved model
init = True
model_name = "model"
num_classes = 4

with graph.as_default():
    #data_set = tf.placeholder(dtype=tf.string)
    data_path = os.path.join('data','train.tfrecords')
    filename_queue = tf.train.string_input_producer([data_path], num_epochs=1)

    # Placeholders
    X, y = read_queue(filename_queue)
    training = tf.placeholder(dtype=tf.bool)
    
    # create global step for decaying learning rate
    global_step = tf.Variable(0, trainable=False)
    
    learning_rate = tf.train.exponential_decay(starting_rate,                 
                                               global_step, 
                                               steps_per_epoch * epochs_per_decay,
                                               decay_factor,                  
                                               staircase=staircase) 
    
    with tf.name_scope('conv1') as scope:
        # Convolutional layer 1 
        conv1 = tf.layers.conv2d(
            X,                           # Input data
            filters=32,                  # 32 filters
            kernel_size=(5, 5),          # Kernel size: 9x9
            strides=(1, 1),              # Stride: 1
            padding='SAME',              # "same" padding
            activation=None,             # None
            kernel_initializer=tf.truncated_normal_initializer(stddev=5e-2, seed=10),
            kernel_regularizer=tf.contrib.layers.l2_regularizer(scale=lamC),
            name='conv1'                 
        )

        # try batch normalization
        conv1 = tf.layers.batch_normalization(
            conv1,
            axis=-1,
            momentum=0.99,
            epsilon=epsilon,
            center=True,
            scale=True,
            beta_initializer=tf.zeros_initializer(),
            gamma_initializer=tf.ones_initializer(),
            moving_mean_initializer=tf.zeros_initializer(),
            moving_variance_initializer=tf.ones_initializer(),
            training=training,
            name='bn1'
        )

        #apply relu
        conv1_bn_relu = tf.nn.relu(conv1, name='relu1')
      
        if dropout:
            conv1_bn_relu = tf.layers.dropout(conv1_bn_relu, rate=0.1, seed=9, training=training)
    
    
    with tf.name_scope('conv2') as scope:
        # Convolutional layer 1 
        conv2 = tf.layers.conv2d(
            conv1_bn_relu,                           # Input data
            filters=32,                  # 32 filters
            kernel_size=(5, 5),          # Kernel size: 9x9
            strides=(1, 1),              # Stride: 1
            padding='SAME',              # "same" padding
            activation=None,             # None
            kernel_initializer=tf.truncated_normal_initializer(stddev=5e-2, seed=10),
            kernel_regularizer=tf.contrib.layers.l2_regularizer(scale=lamC),
            name='conv2'                 
        )

        # try batch normalization
        bn2 = tf.layers.batch_normalization(
            conv2,
            axis=-1,
            momentum=0.99,
            epsilon=epsilon,
            center=True,
            scale=True,
            beta_initializer=tf.zeros_initializer(),
            gamma_initializer=tf.ones_initializer(),
            moving_mean_initializer=tf.zeros_initializer(),
            moving_variance_initializer=tf.ones_initializer(),
            training=training,
            name='bn2'
        )

        #apply relu
        conv2_bn_relu = tf.nn.relu(bn2, name='relu2')
      
        if dropout:
            conv2_bn_relu = tf.layers.dropout(conv2_bn_relu, rate=0.1, seed=9, training=training)
            
    with tf.name_scope('pool1') as scope:
        # Max pooling layer 1
        pool1 = tf.layers.max_pooling2d(
            conv2_bn_relu,               # Input
            pool_size=(3, 3),            # Pool size: 3x3
            strides=(2, 2),              # Stride: 2
            padding='SAME',              # "same" padding
            name='pool1'
        )
        
        if dropout:
          # dropout at 10%
          pool1 = tf.layers.dropout(pool1, rate=0.1, seed=1, training=training)
          
    with tf.name_scope('conv3') as scope:
        # Convolutional layer 3
        conv3 = tf.layers.conv2d(
            pool1,                       # Input data
            filters=64,                  # 48 filters
            kernel_size=(3, 3),          # Kernel size: 5x5
            strides=(1, 1),              # Stride: 1
            padding='SAME',              # "same" padding
            activation=None,             # None
            kernel_initializer=tf.truncated_normal_initializer(stddev=5e-2, seed=10),
            kernel_regularizer=tf.contrib.layers.l2_regularizer(scale=lamC),
            name='conv3'                 
        )

        # try batch normalization
        bn3 = tf.layers.batch_normalization(
            conv3,
            axis=-1,
            momentum=0.99,
            epsilon=epsilon,
            center=True,
            scale=True,
            beta_initializer=tf.zeros_initializer(),
            gamma_initializer=tf.ones_initializer(),
            moving_mean_initializer=tf.zeros_initializer(),
            moving_variance_initializer=tf.ones_initializer(),
            training=training,
            name='bn3'
        )

        #apply relu
        conv3_bn_relu = tf.nn.relu(bn3, name='relu3')
      
        if dropout:
            conv3_bn_relu = tf.layers.dropout(conv3_bn_relu, rate=0.1, seed=9, training=training)
    
    with tf.name_scope('conv4') as scope:
        # Convolutional layer 4
        conv4 = tf.layers.conv2d(
            conv3_bn_relu,                       # Input data
            filters=64,                  # 48 filters
            kernel_size=(3, 3),          # Kernel size: 5x5
            strides=(1, 1),              # Stride: 1
            padding='SAME',              # "same" padding
            activation=None,             # None
            kernel_initializer=tf.truncated_normal_initializer(stddev=5e-2, seed=10),
            kernel_regularizer=tf.contrib.layers.l2_regularizer(scale=lamC),
            name='conv4'                 
        )

        # try batch normalization
        bn4 = tf.layers.batch_normalization(
            conv4,
            axis=-1,
            momentum=0.99,
            epsilon=epsilon,
            center=True,
            scale=True,
            beta_initializer=tf.zeros_initializer(),
            gamma_initializer=tf.ones_initializer(),
            moving_mean_initializer=tf.zeros_initializer(),
            moving_variance_initializer=tf.ones_initializer(),
            training=training,
            name='bn4'
        )

        #apply relu
        conv4_bn_relu = tf.nn.relu(bn4, name='relu4')
      
        if dropout:
            conv4_bn_relu = tf.layers.dropout(conv4_bn_relu, rate=0.1, seed=9, training=training)
    
    with tf.name_scope('pool2') as scope:
        # Max pooling layer 2
        pool2 = tf.layers.max_pooling2d(
            conv4_bn_relu,               # Input
            pool_size=(3, 3),            # Pool size: 2x2
            strides=(2, 2),              # Stride: 2
            padding='SAME',              # "same" padding
            name='pool2'
        )
        
        if dropout:
          # dropout at 10%
          pool2 = tf.layers.dropout(pool2, rate=0.1, seed=1, training=training)
          
    with tf.name_scope('conv5') as scope:
        # Convolutional layer 5
        conv5 = tf.layers.conv2d(
            pool2,                       # Input data
            filters=96,                  # 64 filters
            kernel_size=(3, 3),          # Kernel size: 5x5
            strides=(1, 1),              # Stride: 1
            padding='SAME',              # "same" padding
            activation=None,             # None
            kernel_initializer=tf.truncated_normal_initializer(stddev=5e-2, seed=10),
            kernel_regularizer=tf.contrib.layers.l2_regularizer(scale=lamC),
            name='conv5'                 
        )

        # try batch normalization
        bn5 = tf.layers.batch_normalization(
            conv5,
            axis=-1,
            momentum=0.99,
            epsilon=epsilon,
            center=True,
            scale=True,
            beta_initializer=tf.zeros_initializer(),
            gamma_initializer=tf.ones_initializer(),
            moving_mean_initializer=tf.zeros_initializer(),
            moving_variance_initializer=tf.ones_initializer(),
            training=training,
            name='bn5'
        )

        #apply relu
        conv5_bn_relu = tf.nn.relu(bn5, name='relu5')
      
        if dropout:
            conv5_bn_relu = tf.layers.dropout(conv5_bn_relu, rate=0.1, seed=9, training=training)
    
    with tf.name_scope('conv6') as scope:
        # Convolutional layer 3
        conv6 = tf.layers.conv2d(
            conv5_bn_relu,                       # Input data
            filters=96,                  # 48 filters
            kernel_size=(3, 3),          # Kernel size: 5x5
            strides=(1, 1),              # Stride: 1
            padding='SAME',              # "same" padding
            activation=None,             # None
            kernel_initializer=tf.truncated_normal_initializer(stddev=5e-2, seed=10),
            kernel_regularizer=tf.contrib.layers.l2_regularizer(scale=lamC),
            name='conv6'                 
        )

        # try batch normalization
        bn6 = tf.layers.batch_normalization(
            conv6,
            axis=-1,
            momentum=0.99,
            epsilon=epsilon,
            center=True,
            scale=True,
            beta_initializer=tf.zeros_initializer(),
            gamma_initializer=tf.ones_initializer(),
            moving_mean_initializer=tf.zeros_initializer(),
            moving_variance_initializer=tf.ones_initializer(),
            training=training,
            name='bn6'
        )

        #apply relu
        conv6_bn_relu = tf.nn.relu(bn6, name='relu6')
      
        if dropout:
            conv6_bn_relu = tf.layers.dropout(conv6_bn_relu, rate=0.1, seed=9, training=training)
            
    with tf.name_scope('pool3') as scope:
        # Average pooling layer 3
        pool3 = tf.layers.max_pooling2d(
            conv6_bn_relu,               # Input
            pool_size=(2, 2),            # Pool size: 2x2
            strides=(2, 2),              # Stride: 2
            padding='SAME',              # "same" padding
            name='pool3'
        )
        
        if dropout:
          # dropout at 10%
          pool3 = tf.layers.dropout(pool3, rate=0.25, seed=1, training=training)
    
    with tf.name_scope('conv7') as scope:
        # Convolutional layer 7
        conv7 = tf.layers.conv2d(
            pool3,                       # Input data
            filters=128,                  # 64 filters
            kernel_size=(3, 3),          # Kernel size: 5x5
            strides=(1, 1),              # Stride: 1
            padding='SAME',              # "same" padding
            activation=None,             # None
            kernel_initializer=tf.truncated_normal_initializer(stddev=5e-2, seed=10),
            kernel_regularizer=tf.contrib.layers.l2_regularizer(scale=lamC),
            name='conv7'                 
        )

        # try batch normalization
        bn7 = tf.layers.batch_normalization(
            conv7,
            axis=-1,
            momentum=0.99,
            epsilon=epsilon,
            center=True,
            scale=True,
            beta_initializer=tf.zeros_initializer(),
            gamma_initializer=tf.ones_initializer(),
            moving_mean_initializer=tf.zeros_initializer(),
            moving_variance_initializer=tf.ones_initializer(),
            training=training,
            name='bn7'
        )

        #apply relu
        conv7_bn_relu = tf.nn.relu(bn7, name='relu7')
      
        if dropout:
            conv7_bn_relu = tf.layers.dropout(conv7_bn_relu, rate=0.1, seed=9, training=training)
    
    with tf.name_scope('conv8') as scope:
        # Convolutional layer 8
        conv8 = tf.layers.conv2d(
            conv7_bn_relu,                       # Input data
            filters=128,                  # 48 filters
            kernel_size=(3, 3),          # Kernel size: 5x5
            strides=(1, 1),              # Stride: 1
            padding='SAME',              # "same" padding
            activation=None,             # None
            kernel_initializer=tf.truncated_normal_initializer(stddev=5e-2, seed=10),
            kernel_regularizer=tf.contrib.layers.l2_regularizer(scale=lamC),
            name='conv8'                 
        )

        # try batch normalization
        bn8 = tf.layers.batch_normalization(
            conv8,
            axis=-1,
            momentum=0.99,
            epsilon=epsilon,
            center=True,
            scale=True,
            beta_initializer=tf.zeros_initializer(),
            gamma_initializer=tf.ones_initializer(),
            moving_mean_initializer=tf.zeros_initializer(),
            moving_variance_initializer=tf.ones_initializer(),
            training=training,
            name='bn8'
        )

        #apply relu
        conv8_bn_relu = tf.nn.relu(bn8, name='relu8')
      
        if dropout:
            conv8_bn_relu = tf.layers.dropout(conv8_bn_relu, rate=0.1, seed=9, training=training)
            
    with tf.name_scope('pool4') as scope:
        # Average pooling layer 4
        pool4 = tf.layers.max_pooling2d(
            conv8_bn_relu,               # Input
            pool_size=(2, 2),            # Pool size: 2x2
            strides=(2, 2),              # Stride: 2
            padding='SAME',              # "same" padding
            name='pool4'
        )
        
        if dropout:
          # dropout at 10%
          pool4 = tf.layers.dropout(pool4, rate=0.25, seed=1, training=training)
    
    # Flatten output
    with tf.name_scope('flatten') as scope:
        flat_output = tf.contrib.layers.flatten(pool4)

        # dropout at 10%
        flat_output = tf.layers.dropout(flat_output, rate=0.1, seed=5, training=training)
   
    # Fully connected layer 1
    with tf.name_scope('fc1') as scope:
        fc1 = tf.layers.dense(
            flat_output,                 # input
            1024,                        # 2048 hidden units
            activation=None,             # None
            kernel_initializer=tf.variance_scaling_initializer(scale=2, seed=4),
            bias_initializer=tf.zeros_initializer(),
            kernel_regularizer=tf.contrib.layers.l2_regularizer(scale=lamF),
            name="fc1"
        )
        
        bn_fc1 = tf.layers.batch_normalization(
            fc1,
            axis=-1,
            momentum=0.9,
            epsilon=epsilon,
            center=True,
            scale=True,
            beta_initializer=tf.zeros_initializer(),
            gamma_initializer=tf.ones_initializer(),
            moving_mean_initializer=tf.zeros_initializer(),
            moving_variance_initializer=tf.ones_initializer(),
            training=training,
            name='bn_fc1'
        )
        
        fc1_relu = tf.nn.relu(bn_fc1, name='fc1_relu')
      
        # dropout at 25%
        fc1_relu = tf.layers.dropout(fc1_relu, rate=0.5, seed=10, training=training)

    # Fully connected layer 2
    with tf.name_scope('fc2') as scope:
        fc2 = tf.layers.dense(
            fc1_relu,                     # input
            1024,                       # 1024 hidden units
            activation=None,            # None
            kernel_initializer=tf.variance_scaling_initializer(scale=2, seed=5),
            bias_initializer=tf.zeros_initializer(),
            kernel_regularizer=tf.contrib.layers.l2_regularizer(scale=lamF),
            name="fc2"
        )
        
        bn_fc2 = tf.layers.batch_normalization(
            fc2,
            axis=-1,
            momentum=0.9,
            epsilon=epsilon,
            center=True,
            scale=True,
            beta_initializer=tf.zeros_initializer(),
            gamma_initializer=tf.ones_initializer(),
            moving_mean_initializer=tf.zeros_initializer(),
            moving_variance_initializer=tf.ones_initializer(),
            training=training,
            name='bn_fc2'
        )
        
        fc2_relu = tf.nn.relu(bn_fc2, name='fc2_relu')
        
        # dropout at 10%
        fc2_relu = tf.layers.dropout(fc2_relu, rate=0.5, seed=11, training=training)

    # Output layer
    logits = tf.layers.dense(
        fc2_relu,                      # input
        num_classes,                 # One output unit per category
        activation=None,             # No activation function
        kernel_initializer=tf.variance_scaling_initializer(scale=1, seed=6),
        bias_initializer=tf.zeros_initializer(),
        name="logits"
    )
    
    # Mean cross-entropy
    mean_ce = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits))
    loss = mean_ce + tf.losses.get_regularization_loss()
    
    # Adam optimizer
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    
    # Minimize cross-entropy
    train_op = optimizer.minimize(loss, global_step=global_step)

    # Compute predictions and accuracy
    predictions = tf.argmax(logits, axis=1, output_type=tf.int64)
    is_correct = tf.equal(y, predictions)
    accuracy = tf.reduce_mean(tf.cast(is_correct, dtype=tf.float32))
    
    # add this so that the batch norm gets run
    extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    
    # Create summary hooks
    tf.summary.scalar('accuracy', accuracy)
    tf.summary.scalar('cross_entropy', mean_ce)
    tf.summary.scalar('learning_rate', learning_rate)
    tf.summary.scalar('loss', loss)
    
    # Merge all the summaries and write them out to /tmp/mnist_logs (by default)
    merged = tf.summary.merge_all()

In [ ]:


## CONFIGURE OPTIONS
init = True                   # whether to initialize the model or use a saved version
crop = False                  # do random cropping of images?

meta_data_every = 5
log_to_tensorboard = True
print_every = 3                # how often to print metrics
checkpoint_every = 1           # how often to save model in epochs
use_gpu = True                 # whether or not to use the GPU
print_metrics = False          # whether to print or plot metrics, if False a plot will be created and updated every epoch

# Placeholders for metrics
if init:
    valid_acc_values = []
    valid_cost_values = []
    train_acc_values = []
    train_cost_values = []
    train_lr_values = []
    train_loss_values = []
    

if use_gpu:
    config = tf.ConfigProto()
    config.gpu_options.allocator_type = 'BFC'
    config.gpu_options.per_process_gpu_memory_fraction = 0.7
else:
    config = tf.ConfigProto(device_count = {'GPU': 0})


In [ ]:
# train the model
with tf.Session(graph=graph, config=config) as sess:
    if log_to_tensorboard:
        train_writer = tf.summary.FileWriter('./logs/tr_' + model_name, sess.graph)
        test_writer = tf.summary.FileWriter('./logs/te_' + model_name)
    
    if not print_metrics:
        # create a plot to be updated as model is trained
        f, ax = plt.subplots(1,3,figsize=(20,5))
    
    # create the saver
    saver = tf.train.Saver()
    
    # If the model is new initialize variables, else restore the session
    if init:
        sess.run(tf.global_variables_initializer())
    else:
        saver.restore(sess, './model/'+model_name+'.ckpt')

    # Set seed
    np.random.seed(0)
    
    print("Training", model_name, "...")
    
    # Train several epochs
    for epoch in range(epochs):
        # Accuracy values (train) after each batch
        batch_acc = []
        batch_cost = []
        batch_loss = []
        batch_lr = []
        
        # only log run metadata once per epoch
        write_meta_data = False
        run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
        run_metadata = tf.RunMetadata()
        _, _, summary, acc_value, cost_value, loss_value, step, lr = sess.run([train_op, extra_update_ops, merged, accuracy, mean_ce, loss, global_step, learning_rate],
                feed_dict = {
                  training: True,
                  #data_set: 'train.tfrecords',
                },
                options=run_options,
                run_metadata=run_metadata)
        
        # Save accuracy (current batch)
        batch_acc.append(acc_value)
        batch_cost.append(cost_value)
        batch_lr.append(lr)
        batch_loss.append(loss_value)

        # write the summary
        train_writer.add_run_metadata(run_metadata, 'step %d' % step)
        train_writer.add_summary(summary, step)
        write_meta_data = False
                
        continue
        
        for X_batch, y_batch in get_batches(X_tr, y_tr, batch_size, distort=True):
            if write_meta_data and log_to_tensboard:
                # create the metadata
                run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
                run_metadata = tf.RunMetadata()
            
                # Run training and evaluate accuracy
                _, _, summary, acc_value, cost_value, loss_value, step, lr = sess.run([train_op, extra_update_ops, merged, accuracy, mean_ce, loss, global_step, learning_rate]
                  #, feed_dict={
                  #  X: X_batch,
                  #  y: y_batch,
                  #  training: True
                #}
                ,
                options=run_options,
                run_metadata=run_metadata)

                # Save accuracy (current batch)
                batch_acc.append(acc_value)
                batch_cost.append(cost_value)
                batch_lr.append(lr)
                batch_loss.append(loss_value)
  
                # write the summary
                train_writer.add_run_metadata(run_metadata, 'step %d' % step)
                train_writer.add_summary(summary, step)
                write_meta_data = False
                
            else:
                # Run training without meta data
                _, _, summary, acc_value, cost_value, loss_value, step, lr = sess.run([train_op, extra_update_ops, merged, accuracy, mean_ce, loss, global_step, learning_rate], feed_dict={
                    X: X_batch,
                    y: y_batch,
                    training: True
                })

                # Save accuracy (current batch)
                batch_acc.append(acc_value)
                batch_cost.append(cost_value)
                batch_lr.append(lr)
                batch_loss.append(loss_value)
                
                # write the summary
                if log_to_tensorboard:
                    train_writer.add_summary(summary, step)

        # save checkpoint every nth epoch
        if(epoch % checkpoint_every == 0):
            print("Saving checkpoint")
            # save the model
            save_path = saver.save(sess, './model/'+model_name+'.ckpt')
    
            # Now that model is saved set init to false so we reload it
            init = False
        
        # init batch arrays
        batch_cv_acc = []
        batch_cv_cost = []
        batch_cv_loss = []
        
        # Evaluate validation accuracy with batches so as to not crash the GPU
        for X_batch, y_batch in get_batches(X_cv, y_cv, batch_size, distort=False):
            summary, valid_acc, valid_cost, valid_loss = sess.run([merged, accuracy, mean_ce, loss], feed_dict={
                X: X_batch,
                y: y_batch,
                training: False
            })

            batch_cv_acc.append(valid_acc)
            batch_cv_cost.append(valid_cost)
            batch_cv_loss.append(valid_loss)

        # Write average of validation data to summary logs
        if log_to_tensorboard:
            summary = tf.Summary(value=[tf.Summary.Value(tag="accuracy", simple_value=np.mean(batch_cv_acc)),tf.Summary.Value(tag="cross_entropy", simple_value=np.mean(batch_cv_cost)),])
            test_writer.add_summary(summary, step)
            step += 1
            
        # take the mean of the values to add to the metrics
        valid_acc_values.append(np.mean(batch_cv_acc))
        valid_cost_values.append(np.mean(batch_cv_cost))
        train_acc_values.append(np.mean(batch_acc))
        train_cost_values.append(np.mean(batch_cost))
        train_lr_values.append(np.mean(batch_lr))
        train_loss_values.append(np.mean(batch_loss))
        
        if print_metrics:
            # Print progress every nth epoch to keep output to reasonable amount
            if(epoch % print_every == 0):
                print('Epoch {:02d} - step {} - cv acc: {:.3f} - train acc: {:.3f} (mean) - cv cost: {:.3f} - lr: {:.5f}'.format(
                    epoch, step, np.mean(batch_cv_acc), np.mean(batch_acc), np.mean(batch_cv_cost), lr
                ))
        else:
            # update the plot
            ax[0].cla()
            ax[0].plot(valid_acc_values, color="red", label="Validation")
            ax[0].plot(train_acc_values, color="blue", label="Training")
            ax[0].set_title('Validation accuracy: {:.4f} (mean last 3)'.format(np.mean(valid_acc_values[-4:])))
            
            # since we can't zoom in on plots like in tensorboard, scale y axis to give a decent amount of detail
            if np.mean(valid_acc_values[-3:]) > 0.90:
                ax[0].set_ylim([0.80,1.0])
            elif np.mean(valid_acc_values[-3:]) > 0.85:
                ax[0].set_ylim([0.75,1.0])
            elif np.mean(valid_acc_values[-3:]) > 0.75:
                ax[0].set_ylim([0.65,1.0])
            elif np.mean(valid_acc_values[-3:]) > 0.65:
                ax[0].set_ylim([0.55,1.0])
            elif np.mean(valid_acc_values[-3:]) > 0.55:
                ax[0].set_ylim([0.45,1.0])           
            
            ax[0].set_xlabel('Epoch')
            ax[0].set_ylabel('Accuracy')
            ax[0].legend()
            
            ax[1].cla()
            ax[1].plot(valid_cost_values, color="red", label="Validation")
            ax[1].plot(train_cost_values, color="blue", label="Training")
            ax[1].set_title('Validation xentropy: {:.3f} (mean last 3)'.format(np.mean(valid_cost_values[-4:])))
            ax[1].set_xlabel('Epoch')
            ax[1].set_ylabel('Cross Entropy')
            ax[1].set_ylim([0,2.0])
            ax[1].legend()
            
            ax[2].cla()
            ax[2].plot(train_lr_values)
            ax[2].set_title("Learning rate: {:.6f}".format(np.mean(train_lr_values[-1:])))
            ax[2].set_xlabel("Epoch")
            ax[2].set_ylabel("Learning Rate")
            
            display.display(plt.gcf())
            display.clear_output(wait=True)
            
        # Print data every 50th epoch so I can write it down to compare models
        if (not print_metrics) and (epoch % 50 == 0) and (epoch > 1):
            if(epoch % print_every == 0):
                print('Epoch {:02d} - step {} - cv acc: {:.4f} - train acc: {:.3f} (mean) - cv cost: {:.3f} - lr: {:.5f}'.format(
                    epoch, step, np.mean(batch_cv_acc), np.mean(batch_acc), np.mean(batch_cv_cost), lr
                ))  
            
    # print results of last epoch
    print('Epoch {} - cv acc: {:.4f} - train acc: {:.4f} (mean) - cv cost: {:.3f}'.format(
                epochs, np.mean(batch_cv_acc), np.mean(batch_acc), np.mean(batch_cv_cost)
            ))
    
    # save the session
    save_path = saver.save(sess, './model/'+model_name+'.ckpt')
    
    # init the test data array
    test_acc_values = []
    
    # Check on the test data
    for X_batch, y_batch in get_batches(X_te, y_te, batch_size, distort=False):
        test_accuracy = sess.run(accuracy, feed_dict={
            X: X_batch,
            y: y_batch,
            training: False
        })
        test_acc_values.append(test_accuracy)
    
    # average test accuracy across batches
    test_acc = np.mean(test_acc_values)
    
# show the plot
plt.show()

# print results of last epoch
print('Epoch {} - cv acc: {:.4f} - train acc: {:.4f} (mean) - cv cost: {:.3f}'.format(
      epochs, np.mean(batch_cv_acc), np.mean(batch_acc), np.mean(batch_cv_cost)
    ))
    
# print test accuracy
print("Convolutional network accuracy (test set):",test_acc, " Validation set:", valid_acc_values[-1])